In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window,WindowSpec
import numpy as np


In [2]:
spark = SparkSession.builder.appName('temp').getOrCreate()

In [3]:
df = spark.read.format('csv').load('/FileStore/tables/Machine_learning/data_science_screening_exercise_orders.csv', inferschema = True, header = True)

In [4]:
#df.show()
#df.describe().show()

In [5]:
ordered_data = df.orderBy('customer_id')
ranked_data = ordered_data.withColumn("rank",rank().over(Window.partitionBy("customer_id").orderBy(("date"))))
ranked_data.show()

In [6]:
grouped = ranked_data.groupBy("customer_id").agg(collect_list(struct("date", "rank")).alias("tmp"))
grouped_data = grouped.select("customer_id", sort_array("tmp")["rank"].alias("customer_ranks"))
grouped_data.show()

In [7]:
grouped_data = grouped.select("customer_id", sort_array("tmp")["rank"].alias("customer_ranks"))

In [8]:
result = (grouped_data.drop('customer_id'))
result.show()

In [9]:
result1 = result.collect()
result1